## Collect and pre-process data for NBA regression analysis
**What data are we exploring?**<br>
I am interested in sports analytics, and my favorite sport is basketball, so I used player performance stats from the last 5 NBA seasons. I wrote custom functions that leverage player names and IDs to pull stats from the NBA_API. After executing the functions and merging the returned dataframes, we have a dataset that contains 476 rows and 21 columns. The columns represent standard NBA performance metrics such as:
- total points
- total steals
- total assists
- etc.<br>

This notebook will collect and pre-process the data for regression analysis.


### 1. Import libraries required for collecting and pre-processing

Below is a list of libraries used to collect and manipulate the data.

In [2]:
from data_collection import * #custom created functions
import numpy as np
import pandas as pd
import sys
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.simplefilter(action='ignore', category=FutureWarning)

### 2. Collecting data

Use the custom fucntions to get contract, player IDs, and stats from the last 5 years.<br>
If necessary insert a cell and use help(function) to print the associated docString for a function.<br>
- ex. help(get_contract_data) prints the following:<br>

> Help on function get_contract_data in module data_collection:<br>
<br>
get_contract_data( )<br>
&emsp;Reads the raw 'player_contract_data' file and executes the following<br>
&emsp;manipulations:<br>
&emsp;&emsp;- Converts the 'Player' column to lowercase<br>
&emsp;&emsp;- Replaces accented characters with their english versions.<br>
&emsp;&emsp;- Splits the 'Player' column into 'First_Name' and 'Last_Name' columns.<br>
&emsp;&emsp;- Drops the original 'Player' column since we no longer need it.<br>
&emsp;&emsp;- Rename the '2023-24' column to something more representative<br>
&emsp;&emsp;- Drop the extra Reggie Bullock row<br>
<br>
&emsp;Args:<br>
&emsp;&emsp;None<br>
<br>
&emsp;Returns:<br>
&emsp;&emsp;Dataframe: Player with their associated contract values.

In [3]:
#load player contract data and view sample output
contract_data = get_contract_data()
contract_data.head()

,Current_Contract,First_Name,Last_Name
0,51915615.0,stephen,curry
1,47649433.0,kevin,durant
2,47607350.0,nikola,jokic
3,47607350.0,joel,embiid
4,47607350.0,lebron,james


In [4]:
#load PlayerID data and view sample output
id_data = get_players_with_ids()
id_data.head()

,First_Name,Last_Name,PlayerID
0,alaa,abdelnaby,76001
1,zaid,abdul-aziz,76002
2,kareem,abdul-jabbar,76003
3,mahmoud,abdul-rauf,51
4,tariq,abdul-wahad,1505


In [5]:
#save a copy
id_data.to_csv('../data/player_ids.csv', index = False)

Now that we have the players contract value and IDs we can merge these two dataframes on a first and last name.

In [6]:
# Left join to get IDs added to the contracts dataframe
contract_with_ids = pd.merge(contract_data, id_data, 
                             left_on=['First_Name', 'Last_Name'], 
                             right_on=['First_Name', 'Last_Name'], how='left')

contract_with_ids.head()

,Current_Contract,First_Name,Last_Name,PlayerID
0,51915615.0,stephen,curry,201939
1,47649433.0,kevin,durant,201142
2,47607350.0,nikola,jokic,203999
3,47607350.0,joel,embiid,203954
4,47607350.0,lebron,james,2544


In the next cell I'll use one of the custom functions to make a call to the NBA_API. Since APIs often implement rate limits which prevents excessive use the function will request player performance stats in batches of 50. The time to execute the cell below depends on the server load at the time the cell runs. Depending on the server load at time of execution the cell below can take several minutes to run and may timeout due to a connectionerror. If that occurs wait a few seconds and re-run the cell, if it still doesn't work a backup copy can be loaded by uncommenting the cell below.

In [7]:
#get career stats from a players last five years in batches of 50
stats_df = get_analysis_df(50,contract_with_ids)
stats_df.head()

,PLAYER_ID,GP,GS,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,STL,BLK,TOV,PF,PTS
0,2544,278,277,9795.0,2872,5661,645,1862,1225,1717,280,1972,2212,321,185,1000,491,7614
1,76526,283,116,6027.0,987,2279,18.0,80.0,708,832,104,264,1186,179,18,442.0,434,2700
2,101108,322,322,10293.0,1832,3890,501,1377,899,1025,136,1324,2793,536,90,736,730,5064
3,200752,310,58,6515.0,1172,2636,357,1015,426,513,254,1286,497,178,144,351,540,3127
4,200768,287,276,9764.0,1370,3250,703,1942,859,1006,184,1174,2090,342,109,757,819,4302


In [8]:
#save a copy
stats_df.to_csv('../data/player_stats.csv', index = False)

Now that we have player performance data we can merge this to the "contract_with_ids" dataframe.

In [9]:
# Left join to get stats added to the contracts dataframe
contract_with_ids.rename(columns={'PlayerID': 'PLAYER_ID'}, inplace=True)
analysis_df = pd.merge(contract_with_ids, stats_df, on=['PLAYER_ID'], 
                       how='left')
analysis_df.head()

,Current_Contract,First_Name,Last_Name,PLAYER_ID,GP,GS,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,STL,BLK,TOV,PF,PTS
0,51915615.0,stephen,curry,201939,257,257,8774.0,2417,5144,1261,3049,1183,1287,151,1265,1513,311,78,806,543,7278
1,47649433.0,kevin,durant,201142,184,181,6548.0,1868,3442,386,949,1196,1315,76,1204,1016,141,231,623,381,5318
2,47607350.0,nikola,jokic,203999,368,368,12126.0,3323,5903,409,1199,1597,1930,972,3220,2953,484,257,1224,1007,8652
3,47607350.0,joel,embiid,203954,300,300,9825.0,2817,5561,353,1040,2659,3206,673,2796,1089,284,467,984,894,8646
4,47607350.0,lebron,james,2544,278,277,9795.0,2872,5661,645,1862,1225,1717,280,1972,2212,321,185,1000,491,7614


In [10]:
#save a copy
analysis_df.to_csv('../data/analysis_df.csv', index = False)

### 3. Assess the dataframe

In the section of the notebook I will do the following: 
- Check for and handle missing values
- Check for duplicates
- Review the data types

In [11]:
analysis_df.isnull().sum()

Current_Contract     1
First_Name           0
Last_Name            0
PLAYER_ID            0
GP                  50
GS                  50
MIN                 50
FGM                 50
FGA                 50
FG3M                50
FG3A                50
FTM                 50
FTA                 50
OREB                50
DREB                50
AST                 50
STL                 50
BLK                 50
TOV                 50
PF                  50
PTS                 50
dtype: int64

One player in the dataset does not have a contract, so we will remove that player. Also, there are 50 rookies who are playing their first season this year, however the 2023-2024 season was removed from the dataset because the season is still in progress and contains incomplete data. These players will also be removed from the dataset. We should have 425 rows after dropping rookies and the veteran player without a contract.

In [12]:
filtered_df = analysis_df.dropna(subset=['Current_Contract'])
rookie_filter = filtered_df.dropna(subset= ['GP']).reset_index(drop= True)

In [13]:
rookie_filter.shape

(425, 21)

In [14]:
#confirm that we have addressed the null values
rookie_filter.isnull().sum()

Current_Contract    0
First_Name          0
Last_Name           0
PLAYER_ID           0
GP                  0
GS                  0
MIN                 0
FGM                 0
FGA                 0
FG3M                0
FG3A                0
FTM                 0
FTA                 0
OREB                0
DREB                0
AST                 0
STL                 0
BLK                 0
TOV                 0
PF                  0
PTS                 0
dtype: int64

There are times when a player is traded from one team to another within the same season. The contract data file contains a row for each player and team they played for, so when trades occur we get more than one row for the traded player. The cell below will check for trades by assessing duplicate PLAYER_IDs.

In [15]:
duplicates = rookie_filter['PLAYER_ID'].duplicated()
dupe_list =  rookie_filter['PLAYER_ID'][duplicates].tolist()

# Display the rows where "PLAYER_ID" is a duplicate
print("Duplicate PLAYER_IDs:",dupe_list)

Duplicate PLAYER_IDs: ['201580', '1626166', '1627884', '1630563', '1628464', '1630205']


Six players were traded, before we drop the duplicate rows lets ensure they're duplicate.

In [16]:
filtered_df = rookie_filter[rookie_filter['PLAYER_ID'].isin(dupe_list)]
sorted_result = filtered_df.sort_values(by='First_Name')
sorted_result

,Current_Contract,First_Name,Last_Name,PLAYER_ID,GP,GS,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,STL,BLK,TOV,PF,PTS
201,8519706.0,cameron,payne,1626166,272,55,5571.0,1047,2475,351,958,298,362,88,618,1209,182,66,428,511,2743
347,8519706.0,cameron,payne,1626166,272,55,5571.0,1047,2475,351,958,298,362,88,618,1209,182,66,428,511,2743
192,8670002.0,daniel,theis,1628464,108,56,2172.0,344,670,70,236,108,168,132,368,100,44,74,98,236,866
402,8670002.0,daniel,theis,1628464,108,56,2172.0,344,670,70,236,108,168,132,368,100,44,74,98,236,866
271,5379706.0,derrick,jones jr.,1627884,288,59,4902.0,587,1154,137,418,295,399,317,532,175,145,194,142,509,1606
357,5379706.0,derrick,jones jr.,1627884,288,59,4902.0,587,1154,137,418,295,399,317,532,175,145,194,142,509,1606
313,4368030.0,javale,mcgee,201580,265,49,3557.0,848,1381,13,48,288,429,440,936,139,68,254,313,550,1997
336,4368030.0,javale,mcgee,201580,265,49,3557.0,848,1381,13,48,288,429,440,936,139,68,254,313,550,1997
245,6105941.0,joshua,primo,1630563,108,32,2116.0,220,592,90,298,102,136,68,184,198,42,50,136,174,632
401,6105941.0,joshua,primo,1630563,108,32,2116.0,220,592,90,298,102,136,68,184,198,42,50,136,174,632


The cell above shows that we can drop duplicates without losing information. After dropping dupes we should have 419 rows.

In [17]:
deduplicated_df = rookie_filter.drop_duplicates(subset='PLAYER_ID').reset_index(drop= True)
deduplicated_df.shape

(419, 21)

In [18]:
#review data types
deduplicated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Current_Contract  419 non-null    float64
 1   First_Name        419 non-null    object 
 2   Last_Name         419 non-null    object 
 3   PLAYER_ID         419 non-null    object 
 4   GP                419 non-null    object 
 5   GS                419 non-null    object 
 6   MIN               419 non-null    float64
 7   FGM               419 non-null    object 
 8   FGA               419 non-null    object 
 9   FG3M              419 non-null    object 
 10  FG3A              419 non-null    object 
 11  FTM               419 non-null    object 
 12  FTA               419 non-null    object 
 13  OREB              419 non-null    object 
 14  DREB              419 non-null    object 
 15  AST               419 non-null    object 
 16  STL               419 non-null    object 
 1

In the output I above I see the first four columns have the correct data type, however with the exception of 'MIN' the remaining columns need to be converted to int data type.

In [19]:
# Convert object columns to int starting from the fourth column
for col in deduplicated_df.columns[4:]:
    if deduplicated_df[col].dtype == 'float64':
        pass
    elif deduplicated_df[col].dtype == 'object':
        deduplicated_df[col] = pd.to_numeric(deduplicated_df[col]).astype(int)

In [20]:
#validate that columns were converted correctly
deduplicated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Current_Contract  419 non-null    float64
 1   First_Name        419 non-null    object 
 2   Last_Name         419 non-null    object 
 3   PLAYER_ID         419 non-null    object 
 4   GP                419 non-null    int32  
 5   GS                419 non-null    int32  
 6   MIN               419 non-null    float64
 7   FGM               419 non-null    int32  
 8   FGA               419 non-null    int32  
 9   FG3M              419 non-null    int32  
 10  FG3A              419 non-null    int32  
 11  FTM               419 non-null    int32  
 12  FTA               419 non-null    int32  
 13  OREB              419 non-null    int32  
 14  DREB              419 non-null    int32  
 15  AST               419 non-null    int32  
 16  STL               419 non-null    int32  
 1

In [21]:
#save a backup copy for use in case API is down
deduplicated_df.to_csv('../data/nba_regression_analysis_df.csv', index = False)